In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


In [2]:
spark = SparkSession.builder.appName('pract').getOrCreate()


22/06/09 14:02:13 WARN Utils: Your hostname, fduran-ThinkPad-T490 resolves to a loopback address: 127.0.1.1; using 192.168.0.224 instead (on interface wlp0s20f3)
22/06/09 14:02:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/09 14:02:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_pyspark = spark.read.json('Sellers.json')


In [4]:
df_pyspark.printSchema()

root
 |-- body: struct (nullable = true)
 |    |-- address: struct (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- state: string (nullable = true)
 |    |-- buyer_reputation: struct (nullable = true)
 |    |    |-- tags: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- country_id: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- logo: string (nullable = true)
 |    |-- nickname: string (nullable = true)
 |    |-- permalink: string (nullable = true)
 |    |-- points: long (nullable = true)
 |    |-- registration_date: string (nullable = true)
 |    |-- seller_reputation: struct (nullable = true)
 |    |    |-- level_id: string (nullable = true)
 |    |    |-- power_seller_status: string (nullable = true)
 |    |    |-- transactions: struct (nullable = true)
 |    |    |    |-- canceled: long (nullable = true)
 |    |    |    |-- completed: long (nullable = true)
 |    |    |    |-- period: s

In [23]:
df = df_pyspark.withColumn('body', F.array(F.expr('body')))

In [35]:
df = df_pyspark.selectExpr(
    'body.site_id AS SiteId',
    'body.id as SellerId',
    'body.nickname as SellerNickname',
    'body.points as SellerPoints')

In [36]:
df.filter(df.SellerPoints > 0).show()

+------+---------+----------------+------------+
|SiteId| SellerId|  SellerNickname|SellerPoints|
+------+---------+----------------+------------+
|   MPE|298734964|  MARIELATAQUIRE|           2|
|   MPE|520133997| ISABELLADELPOZO|           1|
|   MPE|285674870|ERICKLOPEZUSMSYA|           2|
|   MPE| 48893023|    MARCELASUSAN|           1|
|   MPE|205264135|     GOBR7283790|           1|
+------+---------+----------------+------------+



In [39]:
df.filter(df.SellerPoints == 0).show()

+------+---------+--------------------+------------+
|SiteId| SellerId|      SellerNickname|SellerPoints|
+------+---------+--------------------+------------+
|   MPE|684964436|         PHMO1747353|           0|
|   MPE|685079498|MELISSASUSANAARVA...|           0|
|   MPE|646068761|   YOMIDELGADOSNCHEZ|           0|
|   MPE|685310649|DONATILDONATILDEC...|           0|
|   MPE|685419864|        VANESSAURNER|           0|
|   MPE|685275449|DANIELARUIZRIDRIGUES|           0|
|   MPE|603331827|         COVA1031117|           0|
|   MPE|580279940|        LORDVENCEDOR|           0|
|   MPE|300834652| DANIELAHILARIORAMOS|           0|
|   MPE|270322958|JHONANTHONYCAYLLA...|           0|
|   MPE|684554092|OSORIOCOLQUIJESSM...|           0|
|   MPE|644166286|         GOEL1504737|           0|
|   MPE|685449106|  ISABELSNCHEZMEDINA|           0|
+------+---------+--------------------+------------+



In [71]:
site_ids = df.filter(df.SellerPoints < 0).select("SiteId").distinct().collect()

In [72]:
full_date = date.today().strftime("%Y%m%d")
year = full_date[0:4]
month = full_date[4:6]
day = full_date[6:]
print(year, day, month)

2022 09 06


In [95]:
import os

for site_id in site_ids:
    directorio = os.getcwd() + "/"+ site_id[0]+"/"+year +"/"+ month + "/"+ day
    df.filter((df.SellerPoints == 0) & (df.SiteId  == site_id[0])).write.mode('overwrite').option('header', True).format('csv').save(directorio + "/Cero")
    df.filter((df.SellerPoints > 0)& (df.SiteId == site_id[0])).write.mode('overwrite').option('header', True).format('csv').save(directorio + "/Positivo")
    df.filter((df.SellerPoints < 0) & (df.SiteId == site_id[0])).write.mode('overwrite').option('header', True).format('csv').save(directorio + "/Negativo")
    